In [1]:
# magic command for automatic reload of python modules without needing to restart the notebook
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os
#adding path to the necessary folders
sys.path.append(os.getcwd() + '/..')
sys.path.append(os.getcwd() + '/../UCI_data_preproc/')


import numpy as np
import scipy as sc
import tensorflow as tf
import tensorflow_probability as tfp
tfd=tfp.distributions
tfb=tfp.bijectors
from scipy import io
import time
import matplotlib.pyplot as plt
from sklearn import mixture
import joblib as jbl
import sys
import sklearn
import pandas as pd
from mixture_models import GMCM, GMC
import utils as utl

# adding path to the data folder
from gas import GAS
from power import POWER
from miniboone import MINIBOONE

In [4]:
data_sets = ['GAS','POWER','MINIBOONE']
data_sets_folder=None
assert data_sets_folder, 'Specify the location of UCI dataset!'

n_reps,mini_batch,max_training_iters = 30,50,10000
ll_list = []

density_estimation={} #initializing dict to 
for i in range(len(data_sets)):
    if data_sets[i] == 'GAS':
        gg=GAS(data_sets_folder)
    elif data_sets[i] == 'POWER':
        gg=POWER(data_sets_folder)
    elif data_sets[i] == 'HEPMASS':
        gg=HEPMASS(data_sets_folder)
    elif data_sets[i] == 'MINIBOONE':
        gg=MINIBOONE(data_sets_folder)

    data_trn = gg.trn.x
    data_tst = gg.tst.x
    data_vld = gg.val.x
    data_all = np.concatenate([data_trn,data_tst,data_vld],axis=0).astype('float32')
    print(f'Number of samples (before downsampling) = {data_all.shape[0]}, Number of dimensions = {data_all.shape[1]}')
    
    # gathering extreme points for each dimension (to be included in the training set)
    extreme_points=np.concatenate([np.argmin(data_all,axis=0), np.argmax(data_all,axis=0)])
    
    log_like_test=np.zeros(n_reps)
    for j in range(n_reps):
        data_trn=data_all[extreme_points,:] # initializing training set with extreme points
        np.random.seed(j)
        rand_id=np.random.permutation(data_all.shape[0])
        trn_id,vld_id,tst_id,_=np.split(rand_id,[10000,11000,12000])
        data_trn=np.concatenate((data_trn,data_all[trn_id,:]),axis=0) 
        data_vld=data_all[vld_id,:]
        data_tst=data_all[tst_id,:]

        nsamps,ndims = data_trn.shape
        print(f'Number of training samples (after downsampling) = {data_trn.shape[0]}, Number of dimensions = {ndims}')

        # Defining a preprocessing bijective transform (eg, log transform in this case)  
        min_val = (np.min(data_all)-3*np.std(data_all)).astype('float32')
        log_transform = tfb.Chain([tfb.Shift(shift=min_val),tfb.Exp()])

        # Training GMCM with data transformation
        # Initialing GMCM object and fitting the distribution 
        gmcm_obj=GMCM(ndims, data_transform=log_transform)
        nll_train, nll_vld=gmcm_obj.fit_dist_IFM(data_trn,
                                                 n_comps=40,
                                                 #data_vld=data_vld,
                                                 batch_size=mini_batch,
                                                 max_iters=max_training_iters,
                                                 regularize=True,
                                                 print_interval=500,
                                                 init = 'gmm', 
                                                 optimizer = tf.optimizers.Adam(learning_rate=1E-3))



        log_like_test[j]=np.mean(gmcm_obj.distribution.log_prob(data_tst))
        density_estimation[data_sets[i]]=log_like_test
        print(density_estimation)
#         jbl.dump(density_estimation,'results/DE_results_gmcm_gmm_init_hepmass')



FileNotFoundError: [Errno 2] No such file or directory: '/home/ec2-user/SageMaker/GMCM-TF/real_world_data/gas/ethylene_CO.pickle'

In [7]:
x=None
assert x, 'Specify the location of UCI dataset!'

AssertionError: Specify the location of UCI dataset!